# Updates Summary

## 1. Introduced Class and Dictionary into Brewster
- introducing classes for better organization and management of code.
- Utilized dictionaries for efficient data storage and retrieval, making it easier to index parameters and add new parameters.

## 2. Merged Multinest and MCMC
- Combined the Multinest and MCMC into a unified framework.

## 3. Simplified Cloud Initialization and Retrieval Setup
- Streamlined the process for initializing cloud parameters.
- Reduced complexity in the setup process, automatically generate p0 in MCMC, making it more user-friendly and less error-prone.

## 4. Added Non-uniform Gas Parameterization




In [1]:
import os 
os.chdir("/Users/fw23aao/brewsteretrieval/brewster")
import utils
import test_module
from collections import namedtuple
import numpy as np
import retrieval_run
import settings

# 1. mcmc template

In [2]:
fwhm=700
wavelength_range=[1,2.8]
ndata=1

chemeq=0
gaslist = ['h2o','co','tio','vo','crh','feh','na','k']
# gaslist = ['h2o','co','tio','vo','crh','feh','k','na']
gastype_list=['U','U','U','U','U','N','U','U']

ptype=77  

do_clouds=1
npatches=2

# cloudname = ['power law cloud slab','powerlaw cloud deck']  
# cloudpacth_index=[[1],[1,2]] 
# particle_dis=['hansan','hansan']

cloudname = ['power law cloud slab']  
# cloudname = ['power law cloud slab']  #'Mie scattering cloud deck--Al2O3'
cloudpacth_index=[[1]] 
particle_dis=['hansan']

do_fudge=1
samplemode='mcmc'

instrument_instance = utils.Instrument(fwhm,wavelength_range,ndata)
re_params = utils.Retrieval_params(samplemode,chemeq,gaslist,gastype_list,fwhm,do_fudge,ptype,do_clouds,npatches,cloudname,cloudpacth_index,particle_dis)
model_config_instance = utils.ModelConfig(samplemode)
io_config_instance = utils.IOConfig()






In [3]:
print(instrument_instance)

Instrument: 
------------
- fwhm : 700
- wavelength_range : [1, 2.8]
- ndata : 1 
- wavpoints : None 



In [4]:
print(model_config_instance)

ModelConfig: 
------------
- use_disort : 0
- do_fudge : 1
- malk : 0
- mch4 : 0
- do_bff : 1
- fresh : 0
- xpath : ../Linelists/
- xlist : gaslistRox.dat
- dist : None
- pfile : LSR1835_eqpt.dat

(MCMC): 
------------
- ndim : None
- nwalkers : None
- nburn : 10000
- niter : 30000



In [5]:
print(io_config_instance)

IOConfig: 
------------
- runname : retrieval
- outdir : N/A
- finalout : retrieval.pk1
- chaindump : retrieval_last_nc.pic
- picdump : retrieval_snapshot.pic
- statfile : status_ball_retrieval.txt
- rfile : runtimes_retrieval.dat
- runtest : 1
- make_arg_pickle : 2



In [6]:
print(re_params)

retrieval_param: 
------------
- gas : ['h2o', 'co', 'tio', 'vo', 'crh', 'feh', 'na', 'k']
- refinement_params : ['logg', 'r2d2', 'dlambda', 'tolerance_parameter_1']
- pt : ['gamma', 'Tint', 'alpha', 'lndelta', 'T1', 'T2', 'T3']
- cloud : ['fcld', 'patch 1']
  -- patch 1: ['tau_pcs', 'logp_pcs', 'dp_pcs', 'omega_pcs', 'alpha_pcs']
- added_params: []



In [9]:
re_params.dictionary['refinement_params']['params']['tolerance_parameter_1']['distribution']

['customized', 0]

In [7]:
re_params.dictionary

{'gas': {'h2o': {'gastype': 'U',
   'params': {'log_abund': {'initialization': None,
     'distribution': ['normal', -4.0, 0.5],
     'range': None,
     'prior': None}}},
  'co': {'gastype': 'U',
   'params': {'log_abund': {'initialization': None,
     'distribution': ['normal', -4.0, 0.5],
     'range': None,
     'prior': None}}},
  'tio': {'gastype': 'U',
   'params': {'log_abund': {'initialization': None,
     'distribution': ['normal', -4.0, 0.5],
     'range': None,
     'prior': None}}},
  'vo': {'gastype': 'U',
   'params': {'log_abund': {'initialization': None,
     'distribution': ['normal', -4.0, 0.5],
     'range': None,
     'prior': None}}},
  'crh': {'gastype': 'U',
   'params': {'log_abund': {'initialization': None,
     'distribution': ['normal', -4.0, 0.5],
     'range': None,
     'prior': None}}},
  'feh': {'gastype': 'N',
   'params': {'log_abund': {'initialization': None,
     'distribution': ['normal', -4.0, 0.5],
     'range': None,
     'prior': None},
    'p_

In [8]:
# re_params.dictionary
all_params,all_params_values =utils.get_all_parametres(re_params.dictionary) 
# all_params

### add new parameters

In [9]:
# re_params.added_params("new1")
# print(re_params)

## 1.1 initialize the parameters

In [10]:
io_config_instance.outdir="/beegfs/car/fei/lsr1835/"
io_config_instance.runname='new_test'
io_config_instance.update_dictionary()


model_config_instance.dist= 5.689 
model_config_instance.update_dictionary()

obspec = np.asfortranarray(np.loadtxt("LSR1835_data_realcalib_new_trimmed.dat",dtype='d',unpack='true'))


args_instance = utils.ArgsGen(re_params,model_config_instance,instrument_instance,obspec)

settings.init(args_instance)
args_instance=settings.runargs

# settings.init()
# settings.runargs=utils.args_gen(re_params,model_config_instance,instrument_instance,obspec)


## 1.2  set initialization distribution

In [46]:
# r2d2 = (71492e3)**2. / (model_config_instance.dist * 3.086e+16)**2.
# re_params.dictionary['refinement_params']['params']['r2d2']['distribution']=['normal', r2d2, 0.1*r2d2]

In [22]:
# model_config_instance.ndim=len(all_params)
# model_config_instance.nwalkers=len(all_params)*16
# p0=utils.MC_P0_gen(re_params.dictionary,model_config_instance)


## 1.3 run retrieval

In [18]:
# retrieval_run.brewster_reterieval_run(re_params,model_config_instance,io_config_instance)

# 2. multinest template

In [20]:
fwhm=700
wavelength_range=[1,2.8]
ndata=1


chemeq=0
gaslist = ['h2o','co','tio','vo','crh','feh','na','k']
gastype_list=['U','U','U','U','U','N','U','U']

ptype=77  
do_clouds=1
npatches=2
cloudname = ['power law cloud slab','powerlaw cloud deck']  
cloudpacth_index=[[1],[1,2]] 
# particle_dis=['hansan','log_normal']


do_fudge=1
samplemode='multinest'

instrument_instance = utils.Instrument(fwhm,wavelength_range,ndata)
re_params = utils.Retrieval_params(samplemode,chemeq,gaslist,gastype_list,fwhm,do_fudge,ptype,do_clouds,npatches,cloudname,cloudpacth_index)
model_config_instance = utils.ModelConfig(samplemode)
io_config_instance = utils.IOConfig()

In [21]:
print(instrument_instance)

Instrument: 
------------
- fwhm : 700
- wavelength_range : [1, 2.8]
- ndata : 1 
- wavpoints : None 



In [22]:
print(model_config_instance)

ModelConfig: 
------------
- use_disort : 0
- do_fudge : 1
- malk : 0
- mch4 : 0
- do_bff : 1
- fresh : 0
- xpath : ../Linelists/
- xlist : gaslistRox.dat
- dist : None
- pfile : LSR1835_eqpt.dat

PyMultiNest: 
----------------------
- LogLikelihood: None 
- Prior: None 
- ndim: None 
- nparam: None
- n_clustering_params: None
- wrapped_params: None
- importance_nested_sampling: True
- multimodal: True
- const_efficiency_mode: False
- n_live_points: 400
- evidence_tolerance: 0.5
- sampling_efficiency: 0.8
- n_iter_before_update: 100
- null_log_evidence: -1e+90
- max_modes: 100
- mode_tolerance: -1e+90
- outputfiles_basename: 
- seed: -1
- verbose: True
- resume: True
- context: 0
- write_output: True
- log_zero: -1e+100
- max_iter: 0
- init_MPI: False
- dump_callback: None



In [23]:
print(io_config_instance)

IOConfig: 
------------
- runname : retrieval
- outdir : N/A
- finalout : retrieval.pk1
- chaindump : retrieval_last_nc.pic
- picdump : retrieval_snapshot.pic
- statfile : status_ball_retrieval.txt
- rfile : runtimes_retrieval.dat
- runtest : 1
- make_arg_pickle : 2



In [24]:
print(re_params)

retrieval_param: 
------------
- gas : ['h2o', 'co', 'tio', 'vo', 'crh', 'feh', 'na', 'k']
- refinement_params : ['M', 'R', 'dlambda', 'tolerance_parameter_1']
- pt : ['gamma', 'Tint', 'alpha', 'lndelta', 'T1', 'T2', 'T3']
- cloud : ['fcld', 'patch 1', 'patch 2']
  -- patch 1: ['tau_pcs', 'logp_pcs', 'dp_pcs', 'omega_pcs', 'alpha_pcs', 'logp_pcd', 'dp_pcd', 'omega_pcd', 'alpha_pcd']
  -- patch 2: ['logp_pcd', 'dp_pcd', 'omega_pcd', 'alpha_pcd']
- added_params: []



## 2.1 initialize the parameters

In [25]:
io_config_instance.outdir="/beegfs/car/fei/lsr1835/test/"
io_config_instance.runname='new_test'
io_config_instance.update_dictionary()


model_config_instance.const_efficiency_mode=True
model_config_instance.sampling_efficiency=0.3
model_config_instance.multimodal = False
model_config_instance.log_zero= -1e90
model_config_instance.importance_nested_sampling= False
model_config_instance.evidence_tolerance=0.1
model_config_instance.dist= 5.689 
model_config_instance.update_dictionary()

obspec = np.asfortranarray(np.loadtxt("LSR1835_data_realcalib_new_trimmed.dat",dtype='d',unpack='true'))
settings.init()
settings.runargs=utils.args_gen(re_params,model_config_instance,instrument_instance,obspec)


## 2.2 run retrieval

In [26]:
# retrieval_run.brewster_reterieval_run(re_params,model_config_instance,io_config_instance)